<a href="https://colab.research.google.com/github/nepomucenoc/pyspark_sales/blob/main/Project_sales_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sales Analysis with PySpark: Exploring Agriculture Sales Data

## Description:

This notebook leverages the powerful PySpark library to conduct an extensive analysis of sales data within the agriculture industry. The dataset used in this analysis comprises several columns, including ID, Date, Product, Quantity, Vendor, Source, Country, Transportation Method, Delivery Date, and Total.

### Key Analyses:

1. **Total Sales per Product**: The code calculates and presents the total sales for each product, providing an overview of the product-wise sales performance.

2. **Top-Selling Products**: The notebook identifies and displays the products with the highest sales figures, aiding in the understanding of which products are driving revenue.

3. **Sales Analysis by Country**: It examines the total sales aggregated by country, offering insights into geographical sales trends.

4. **Sales Analysis by Origin**: This analysis focuses on the total sales categorized by the source of origin, shedding light on the influence of the product source on sales.

5. **Sales Analysis by Transport Method**: The code assesses the total sales categorized by the transportation method used, providing insights into the impact of transportation on sales figures.

6. **Sales Analysis Over Time**: This analysis delves into sales trends over time by calculating and presenting the total sales per year and per month. It allows for the identification of seasonal patterns and long-term sales trends.

7. **Sales Analysis by Seller**: The notebook investigates the total sales attributed to each vendor, offering insights into the sales performance of individual sellers.

8. **Analysis of Best-Selling Products Per Month**: It identifies and displays the products with the highest quantity sold for each month. This analysis aids in understanding which products are consistently popular throughout the year.

9. **Analysis of Best-Selling Products Per Year**: This analysis focuses on identifying the best-selling products on an annual basis, providing insights into long-term product performance.

This comprehensive analysis serves as a valuable resource for extracting actionable insights from agricultural sales data, enabling data-driven decision-making and strategic planning in the industry.


In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=21a79332477afcfae1582c96ceada7a62d1358b6e153fa91675c98ed64774a86
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [36]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, desc
from pyspark.sql.functions import year, month

In [20]:
spark = SparkSession.builder.appName("PySparkProject").getOrCreate()

In [23]:
dataset = '/content/drive/MyDrive/Curso_Power_BI/AULA-03_Dashboard-Vendas/Dados_Agricultura_Pedidos_ed.xlsx'
data = pd.read_excel(dataset)
data.head()

,ID,Data,Produto,Quantidade,Vendedor,Origem,País,Método_Transporte,Data_Entrega,Total
0,1,2023-02-13,banana,461,Vendedor 25,Quinta dos Pinhais,Brasil,Caminhão,2023-03-15,0
1,2,2022-08-17,limão,767,Vendedor 15,Fazenda Luz da Lua,Brasil,Caminhão,2022-08-27,1727
2,3,2023-05-30,limão,57,Vendedor 16,Fazenda Luz da Lua,Brasil,Barco,2023-07-03,2264
3,4,2022-07-14,banana,596,Vendedor 43,Recanto Verde,Brasil,Barco,2022-08-05,0
4,5,2022-05-03,limão,754,Vendedor 41,Fazenda Luz da Lua,Brasil,Caminhão,2022-05-16,0


In [24]:
data.shape

(150000, 10)

In [26]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   ID                 150000 non-null  int64         
 1   Data               150000 non-null  datetime64[ns]
 2   Produto            150000 non-null  object        
 3   Quantidade         150000 non-null  int64         
 4   Vendedor           150000 non-null  object        
 5   Origem             150000 non-null  object        
 6   País               150000 non-null  object        
 7   Método_Transporte  150000 non-null  object        
 8   Data_Entrega       150000 non-null  datetime64[ns]
 9   Total              150000 non-null  int64         
dtypes: datetime64[ns](2), int64(3), object(5)
memory usage: 11.4+ MB


In [25]:
df = spark.createDataFrame(data)

In [27]:
df.head()

Row(ID=1, Data=datetime.datetime(2023, 2, 13, 0, 0), Produto='banana', Quantidade=461, Vendedor='Vendedor 25', Origem='Quinta dos Pinhais', País='Brasil', Método_Transporte='Caminhão', Data_Entrega=datetime.datetime(2023, 3, 15, 0, 0), Total=0)

In [28]:
df.show()

+---+-------------------+-------+----------+-----------+------------------+------+-----------------+-------------------+-----+
| ID|               Data|Produto|Quantidade|   Vendedor|            Origem|  País|Método_Transporte|       Data_Entrega|Total|
+---+-------------------+-------+----------+-----------+------------------+------+-----------------+-------------------+-----+
|  1|2023-02-13 00:00:00| banana|       461|Vendedor 25|Quinta dos Pinhais|Brasil|         Caminhão|2023-03-15 00:00:00|    0|
|  2|2022-08-17 00:00:00|  limão|       767|Vendedor 15|Fazenda Luz da Lua|Brasil|         Caminhão|2022-08-27 00:00:00| 1727|
|  3|2023-05-30 00:00:00|  limão|        57|Vendedor 16|Fazenda Luz da Lua|Brasil|            Barco|2023-07-03 00:00:00| 2264|
|  4|2022-07-14 00:00:00| banana|       596|Vendedor 43|     Recanto Verde|Brasil|            Barco|2022-08-05 00:00:00|    0|
|  5|2022-05-03 00:00:00|  limão|       754|Vendedor 41|Fazenda Luz da Lua|Brasil|         Caminhão|2022-05-16 

In [29]:
total_sales_per_product = df.groupBy("Produto").agg(sum("Total").alias("TotalSales"))
total_sales_per_product.show()

+--------+----------+
| Produto|TotalSales|
+--------+----------+
|     uva|  35495246|
|  banana|  43967669|
|   limão|  59030619|
| laranja|  21899642|
|melancia|   5833903|
|    maçã|  45302315|
|   manga|  22807740|
+--------+----------+



In [30]:
top_selling_products = total_sales_per_product.orderBy(desc("TotalSales"))
top_selling_products.show()

+--------+----------+
| Produto|TotalSales|
+--------+----------+
|   limão|  59030619|
|    maçã|  45302315|
|  banana|  43967669|
|     uva|  35495246|
|   manga|  22807740|
| laranja|  21899642|
|melancia|   5833903|
+--------+----------+



In [33]:
# Sales Analysis by Country
sales_pais = df.groupBy("País").agg(sum("Total").alias("TotalVendasPorPaís"))
sales_pais.show()

+--------------+------------------+
|          País|TotalVendasPorPaís|
+--------------+------------------+
|         China|          24721186|
|Estados Unidos|          32650001|
|        Brasil|          98477108|
|        Canadá|           4920215|
|        França|           5075538|
|      Alemanha|           4784230|
| Países Baixos|           5021305|
|     Argentina|           4847367|
|        Rússia|           4904669|
|       Bélgica|           4910281|
| Coreia do Sul|           5033520|
|         Chile|           4977875|
|       Espanha|           4893520|
|Arábia Saudita|           4740448|
|         Índia|           4864602|
|   Reino Unido|           5012362|
|        Itália|           4857888|
|        México|           4833066|
|         Japão|           4811953|
+--------------+------------------+



In [32]:
# Sales Analysis by Origin:
sales_origin = df.groupBy("Origem").agg(sum("Total").alias("TotalVendasPorOrigem"))
sales_origin.show()

+--------------------+--------------------+
|              Origem|TotalVendasPorOrigem|
+--------------------+--------------------+
|Fazenda Campo Sereno|             5833903|
|  Sítio Águas Claras|            17905583|
|Rancho Estrela Ca...|            11301872|
|  Quinta dos Pinhais|            32916831|
|     Refúgio do Vale|            17589663|
|       Recanto Verde|            32950480|
|Fazenda Sol Nascente|            11505868|
|     Terra Prometida|            22644610|
|  Chácara Bela Vista|            22657705|
|  Fazenda Luz da Lua|            59030619|
+--------------------+--------------------+



In [34]:
# Sales Analysis by Transport Method:
sales_transporte = df.groupBy("Método_Transporte").agg(sum("Total").alias("TotalVendasPorTransporte"))
sales_transporte.show()

+-----------------+------------------------+
|Método_Transporte|TotalVendasPorTransporte|
+-----------------+------------------------+
|             Trem|                12016990|
|            Barco|                35278037|
|         Caminhão|               105241703|
|            Avião|                81800404|
+-----------------+------------------------+



In [37]:
# Sales Analysis Over Time:
df = df.withColumn("Ano", year("Data"))
df = df.withColumn("Mês", month("Data"))

vendas_por_ano = df.groupBy("Ano").agg(sum("Total").alias("TotalVendasPorAno"))
vendas_por_ano.show()

vendas_por_mês = df.groupBy("Mês").agg(sum("Total").alias("TotalVendasPorMês"))
vendas_por_mês.show()

+----+-----------------+
| Ano|TotalVendasPorAno|
+----+-----------------+
|2023|        116033736|
|2022|        118303398|
+----+-----------------+

+---+-----------------+
|Mês|TotalVendasPorMês|
+---+-----------------+
| 12|         22989576|
|  1|         22586987|
|  6|         17470470|
|  3|         17979121|
|  5|         17235750|
|  9|         15965449|
|  4|         21486760|
|  8|         19051911|
|  7|         19393913|
| 10|         17831044|
| 11|         19480781|
|  2|         22865372|
+---+-----------------+



In [38]:
# Sales Analysis by Seller:
sales_seller = df.groupBy("Vendedor").agg(sum("Total").alias("TotalVendasPorVendedor"))
sales_seller.show()

+-----------+----------------------+
|   Vendedor|TotalVendasPorVendedor|
+-----------+----------------------+
| Vendedor 7|               4711157|
|Vendedor 25|               4783711|
|Vendedor 43|               4694139|
| Vendedor 4|               4798053|
|Vendedor 39|               4756994|
|Vendedor 31|               4714930|
|Vendedor 30|               4839073|
|Vendedor 29|               4576073|
|Vendedor 36|               4979394|
|Vendedor 18|               4491721|
|Vendedor 32|               4587185|
| Vendedor 3|               4730516|
| Vendedor 6|               4772666|
| Vendedor 9|               4733441|
|Vendedor 33|               4685733|
| Vendedor 8|               4543889|
|Vendedor 48|               4606096|
|Vendedor 34|               4819318|
|Vendedor 47|               4475384|
|Vendedor 16|               4761647|
+-----------+----------------------+
only showing top 20 rows



In [39]:
# Analysis of best-selling products per month:
most_sold_products_per_month = df.groupBy("Mês", "Produto").agg(sum("Quantidade").alias("TotalQuantidade"))
most_sold_products_per_month = most_sold_products_per_month.orderBy("Mês", desc("TotalQuantidade"))
most_sold_products_per_month.show()

+---+--------+---------------+
|Mês| Produto|TotalQuantidade|
+---+--------+---------------+
|  1|    maçã|        1784047|
|  1|  banana|        1652471|
|  1|     uva|        1337403|
|  1|   limão|        1084299|
|  1| laranja|         824073|
|  1|   manga|         441787|
|  1|melancia|         118326|
|  2|    maçã|        1794325|
|  2|  banana|        1655686|
|  2|     uva|        1327754|
|  2|   limão|        1130018|
|  2| laranja|         840687|
|  2|   manga|         449971|
|  2|melancia|         117265|
|  3|  banana|        1690865|
|  3|   limão|        1063169|
|  3|    maçã|         918301|
|  3| laranja|         845772|
|  3|     uva|         668420|
|  3|   manga|         427831|
+---+--------+---------------+
only showing top 20 rows



In [40]:
# Analysis of best-selling products per year:
most_sold_products_per_year = df.groupBy("Ano", "Produto").agg(sum("Quantidade").alias("TotalQuantidade"))
most_sold_products_per_year = most_sold_products_per_year.orderBy("Ano", desc("TotalQuantidade"))
most_sold_products_per_year.show()

+----+--------+---------------+
| Ano| Produto|TotalQuantidade|
+----+--------+---------------+
|2022|   limão|        9348381|
|2022|    maçã|        7218199|
|2022|  banana|        7075755|
|2022|     uva|        5704824|
|2022|   manga|        3636126|
|2022| laranja|        3536242|
|2022|melancia|         949955|
|2023|   limão|        9456938|
|2023|    maçã|        7233703|
|2023|  banana|        7121174|
|2023|     uva|        5653322|
|2023|   manga|        3665168|
|2023| laranja|        3536676|
|2023|melancia|         940376|
+----+--------+---------------+



In [43]:
most_sold_products_per_year.printSchema()

root
 |-- Ano: integer (nullable = true)
 |-- Produto: string (nullable = true)
 |-- TotalQuantidade: long (nullable = true)



In [45]:
import datetime

In [46]:
days_list = [datetime.datetime(2023, 9, 1, 0, 0), datetime.datetime(2023, 9, 2, 0, 0), datetime.datetime(2023, 9, 3, 0, 0), datetime.datetime(2023, 9, 4, 0, 0), datetime.datetime(2023, 9, 5, 0, 0), datetime.datetime(2023, 9, 6, 0, 0), datetime.datetime(2023, 9, 7, 0, 0), datetime.datetime(2023, 9, 8, 0, 0), datetime.datetime(2023, 9, 9, 0, 0), datetime.datetime(2023, 9, 10, 0, 0), datetime.datetime(2023, 9, 11, 0, 0), datetime.datetime(2023, 9, 12, 0, 0), datetime.datetime(2023, 9, 13, 0, 0), datetime.datetime(2023, 9, 14, 0, 0), datetime.datetime(2023, 9, 15, 0, 0), datetime.datetime(2023, 9, 16, 0, 0), datetime.datetime(2023, 9, 17, 0, 0), datetime.datetime(2023, 9, 18, 0, 0), datetime.datetime(2023, 9, 19, 0, 0), datetime.datetime(2023, 9, 20, 0, 0), datetime.datetime(2023, 9, 21, 0, 0), datetime.datetime(2023, 9, 22, 0, 0), datetime.datetime(2023, 9, 23, 0, 0), datetime.datetime(2023, 9, 24, 0, 0), datetime.datetime(2023, 9, 25, 0, 0), datetime.datetime(2023, 9, 26, 0, 0), datetime.datetime(2023, 9, 27, 0, 0), datetime.datetime(2023, 9, 28, 0, 0), datetime.datetime(2023, 9, 29, 0, 0), datetime.datetime(2023, 9, 30, 0, 0)]

In [49]:
days = list(set([date.replace(day=1) for date in days_list]))
days

[datetime.datetime(2023, 9, 1, 0, 0)]